In [ ]:
from wbot import read,play
import numpy as np

plist=read.word_list('possible')
glist=read.word_list('allowed')


big_arr=[]

for i, g in enumerate(glist):
    small_arr=[]
    for j,p in enumerate(plist):
        small_arr+=[play.tiles(p,g)]
    
    big_arr+=[small_arr]
    
    if i%100==0:
        print(i)



big_arr=np.array(big_arr)


np.savez_compressed('all_tiles.npz',data=big_arr)


In [ ]:
from wbot import read,play
import numpy as np

big_arr=np.load('all_tiles.npz')['data']
un_pat=np.unique(big_arr[12489,:,:],axis=0)


g_word=[]
for pat in un_pat:
    
    print(pat)
    
    tgrid,plist_=reduce('salet',pat,big_arr,np.array(plist),np.array(glist))

    x=prob_grid(tgrid)

    guess=glist[np.argmax(x)]
    
    g_word+=[guess]
    
    print(pat,guess)
    
np.save('salet_patterns.npy',un_pat)
np.save('salet_guess.npy',g_word,)
    

In [ ]:
from wbot import read,play
import numpy as np

big_arr=np.load('all_tiles.npz')['data']


In [ ]:



def reduce(guess,tiles,tgrid,plist,glist):
    
    
    g_id=np.where(glist==guess)[0][0]

    w=np.where(np.prod(tgrid[g_id,:,:] == tiles, axis = -1))[0]
    
    return tgrid[:,w,:],plist[w]



def prob_grid(tgrid):
    
    gs,ps,_=tgrid.shape
    
    egrid=np.zeros(gs)
    
    for i in range(gs):
        
        un_pat=np.unique(tgrid[i,:,:],axis=0)
        #prob_row=np.zeros(ps)
        mean_inf=0
        
        for j in range(un_pat.shape[0]):
            
           
            p=len(np.where(np.prod(tgrid[i,:,:] == un_pat[j,:], axis = -1))[0])/ps
            mean_inf+=-p*np.log2(p)
            
            
            
        egrid[i]=mean_inf
    return egrid

def cut_off(w,tgrid,plist,glist,igrid,count):
    ids=[]
    info_list=[]
    for p in plist:
        
        ids=np.where(glist==p)[0][0]
        
        info_list+=[igrid[ids]]
    
    guess=plist[np.argmax(info_list)]
    
    count+=1
    tiles=play.tiles(w,guess)
    if tiles==[2,2,2,2,2]:
        print(w,guess,count)
        return count        
    else:
        tgrid,plist_=reduce(guess,tiles,tgrid,plist,glist)
        
        if len(plist_)==1:
            count+=1
            print(w,plist_[0],count)
            return count
        else:
            return cut_off(w,tgrid,plist_,glist,igrid,count)
        
def manual_cut_off(tgrid,plist,glist,igrid):
    
    print('Possible Answers:', len(plist))
    ids=[]
    info_list=[]
    for p in plist:
        
        ids=np.where(glist==p)[0][0]
        
        info_list+=[igrid[ids]]
    
    guess=plist[np.argmax(info_list)]
    print('Guess:',guess)

    ts=input('Type Tiles')
    if [int(ts[0]),int(ts[1]),int(ts[2]),int(ts[3]),int(ts[4])]==[2,2,2,2,2]:
        print('Done')     
    else:
        tgrid,plist_=reduce(guess,[int(ts[0]),int(ts[1]),int(ts[2]),int(ts[3]),int(ts[4])],tgrid,plist,glist)
        if len(plist_)==1:
            print('Answer:', plist_[0])
        else:
            manual_cut_off(tgrid,plist_,glist,igrid)
                       

In [ ]:
sal_patterns=np.load('salet_patterns.npy')
sal_guess=np.load('salet_guess.npy')

In [ ]:
%%timeit
w='hound'
ts=play.tiles(w,'salet')
print(ts)  
tgrid,plist_=reduce('salet',ts,big_arr,np.array(plist),np.array(glist))

guess=sal_guess[np.where(np.prod(sal_patterns ==ts, axis = -1))[0][0]]
print(guess)
count=1
while len(plist_)>5:
        
    count+=1
    ts=play.tiles(w,guess)
    
    print(ts)
    tgrid,plist_=reduce(guess,ts,tgrid,plist_,np.array(glist))

    x=prob_grid(tgrid)

    guess=glist[np.argmax(x)]
    
    print(guess)
        
    
if count==1:
    x=prob_grid(tgrid)
    count=cut_off(w,tgrid,plist_,np.array(glist),x,count)  
    
else:
    count=cut_off(w,tgrid,plist_,np.array(glist),x,count) 
    
     
        

In [ ]:
res=[]
sal_patterns=np.load('salet_patterns.npy')
sal_guess=np.load('salet_guess.npy')
for i,w in enumerate(plist):
    
    ts=play.tiles(w,'salet')
    
    tgrid,plist_=reduce('salet',ts,big_arr,np.array(plist),np.array(glist))

    guess=sal_guess[np.where(np.prod(sal_patterns ==ts, axis = -1))[0][0]]

    count=1

    while len(plist_)>5:
        
        count+=1

        tgrid,plist_=reduce(guess,play.tiles(w,guess),tgrid,plist_,np.array(glist))

        x=prob_grid(tgrid)

        guess=glist[np.argmax(x)]
        
    
    
    
    if count==1:
        x=prob_grid(tgrid)
        count=cut_off(w,tgrid,plist_,np.array(glist),x,count)  
    
    else:
        count=cut_off(w,tgrid,plist_,np.array(glist),x,count) 
        
    res+=[count]
    print(np.mean(res))
        

In [ ]:
# print('Guess: salet')
ts=input('Type Tiles:')

tgrid,plist_=reduce('salet',[int(ts[0]),int(ts[1]),int(ts[2]),int(ts[3]),int(ts[4])],big_arr,np.array(plist),np.array(glist))

x=prob_grid(tgrid)

guess=glist[np.argmax(x)]

while len(plist_)>5:
    
    print('')
    print('Possible Answers', len(plist_))
    print('Next Guess: ',guess)
    
    ts=input('Type Tiles:')

    tgrid,plist_=reduce(guess,[int(ts[0]),int(ts[1]),int(ts[2]),int(ts[3]),int(ts[4])],tgrid,plist_,np.array(glist))
    
    x=prob_grid(tgrid)

    guess=glist[np.argmax(x)]


    
if count==1:
    x=prob_grid(tgrid)
    count=manual_cut_off(tgrid,plist_,np.array(glist),x)  
    
else:
    count=manual_cut_off(tgrid,plist_,np.array(glist),x) 

